### Connect Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Put your path below

In [2]:
!cd '/content/drive/MyDrive/AIISC-Internship/text-based-object-discovery'

In [3]:
PATH = '/content/drive/MyDrive/AIISC-Internship/text-based-object-discovery'

### Install Required Packages

`Stanza`, Stanford NLP Package benefits from `GPU` so enable it under `View Resources > Change runtime type`

In [4]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-ea4cf3b3-9f34-0aff-d6a6-1c814b85a0a3)


In [5]:
!pip install stanza # for stanford pos tagger
!pip install ftfy regex tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Load Necessary Libraries

We will load the necessary libraries required for generating DAAM outputs for input prompts.

In [6]:
import os
import json
from tqdm import tqdm

from matplotlib import pyplot as plt
import numpy as np

from nltk.corpus import stopwords

from pycocotools.coco import COCO

Download the stopwords for removing stopwords

In [7]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [8]:
import stanza
stanza.download('en')

INFO:stanza:Downloading default packages for language: en (English) ...
INFO:stanza:File exists: /root/stanza_resources/en/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources.


### Load Data

Below, we load the `MS-COCO` annotations to extract the captions to continue further with extracting the objects from each caption.

In [9]:
!wget -c http://images.cocodataset.org/annotations/annotations_trainval2017.zip
!unzip annotations_trainval2017.zip
!rm annotations_trainval2017.zip
!ls annotations

--2023-01-03 10:15:37--  http://images.cocodataset.org/annotations/annotations_trainval2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.216.128.35, 54.231.197.65, 52.216.50.105, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|52.216.128.35|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 252907541 (241M) [application/zip]
Saving to: ‘annotations_trainval2017.zip’

annotations_trainva 100%[===================>] 241.19M  16.0MB/s    in 10s     

2023-01-03 10:15:48 (23.5 MB/s) - ‘annotations_trainval2017.zip’ saved [252907541/252907541]

Archive:  annotations_trainval2017.zip
replace annotations/instances_train2017.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: captions_train2017.json   instances_val2017.json
captions_val2017.json	  person_keypoints_train2017.json
instances_train2017.json  person_keypoints_val2017.json


Now, we load the json file for the train and validation set captions.

In [10]:
with open('annotations/captions_train2017.json') as json_file:
  caption_data_train_file = json.load(json_file)
with open('annotations/captions_val2017.json') as json_file:
  caption_data_val_file = json.load(json_file)

In [11]:
caption_data_train = caption_data_train_file['annotations']
caption_data_val = caption_data_val_file['annotations']

In [12]:
prompts_train = [ann['caption'] for ann in caption_data_train]
prompts_val = [ann['caption'] for ann in caption_data_val]

In [13]:
print('***train captions***\n', '\n'.join(prompts_train[:5]))
print()
print('Number of train captions:', len(prompts_train))
print()
print()
print('***validation captions:***\n', '\n'.join(prompts_val[:5]))
print()
print('Number of train captions:', len(prompts_val))

***train captions***
 A bicycle replica with a clock as the front wheel.
A room with blue walls and a white sink and door.
A car that seems to be parked illegally behind a legally parked car
A large passenger airplane flying through the air.
There is a GOL plane taking off in a partly cloudy sky.

Number of train captions: 591753


***validation captions:***
 A black Honda motorcycle parked in front of a garage.
A Honda motorcycle parked in a grass driveway
An office cubicle with four different types of computers.
A small closed toilet in a cramped space.
Two women waiting at a bench next to a street.

Number of train captions: 25014


Cleaning the prompts. I adopt few ways to clean the prompt:
- Tokenization
- Remove stop words
- Remove non-alphabets
- Keep only nouns
- Lemmatization (to store the object name)

In [14]:
# loads the text processing pipeline
nlp = stanza.Pipeline(lang='en', processors='tokenize,mwt,pos,lemma')

# treebank-specific POS (XPOS) tags to keep, other POS tagged tokens will not be retained
keep_pos_tags = ['NN', 'NNS', 'NNP', 'NNPS']

# Stopwords
stpwords = set(stopwords.words('english'))

# extract parts of speech
def extract_pos(doc):
  parsed_text = list()
  for sent in doc.sentences:
    for wrd in sent.words:
      parsed_text.append((wrd.text, wrd.xpos))
  return parsed_text

# extract lemma
def extract_lemma(doc):
  parsed_text = list()
  for sent in doc.sentences:
    for wrd in sent.words:
      # extract text and lemma
      parsed_text.append((wrd.text, wrd.lemma))
  # return a dataframe
  return parsed_text

def clean_prompt(prompt):
  # tokenizes, lemmatizes and pos tags the prompt
  processed_prompt = nlp(prompt.lower())
  
  # extracts pos tags from the processed_prompt
  pos_tagged_prompt = extract_pos(processed_prompt)

  # lemmatized text
  lemmatized_prompt = extract_lemma(processed_prompt)

  # keep only the noun words, removes stopwords
  fin_prompt = [word for word, pos_tag in pos_tagged_prompt if ((pos_tag in keep_pos_tags) and (word not in stpwords))]
  obj_prompt = [word_lemma[1] for word_pos, word_lemma in zip(pos_tagged_prompt, lemmatized_prompt) if ((word_pos[1] in keep_pos_tags) and (word_lemma[0] not in stpwords))]
  return fin_prompt, obj_prompt

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| pos       | combined |
| lemma     | combined |

INFO:stanza:Use device: gpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Done loading processors!


An example is shown below for the application of `clean_prompt`.

In [15]:
clean_prompt("The fishes are playing in the mountains.")

(['fishes', 'mountains'], ['fish', 'mountain'])

Below, we start processing each prompt and store the objects detected in the captions from train and validation split.

In [16]:
NUM_PROMPTS_INFO_DISPLAY = 500 # After processing how many prompts, some necessary information must be displayed
SAVE_AFTER = 10000 # Save after how many prompts processed

In [19]:
import shutil # Removes directory if already present! CAREFUL!!!!!!!!!!!!!!!!!!
if os.path.exists(os.path.join(PATH, 'Caption-Processing')):
  shutil.rmtree(os.path.join(PATH, 'Caption-Processing'))
os.mkdir(os.path.join(PATH, 'Caption-Processing'))

In [ ]:
try:

  total_objects = set() # Stores the total number of distinct objects detected
  num_objects_detected = list() # Stores number of objects detected after processing some number of prompts iteratively

  print('Starting...')
  print('Captions to be processed:', len(prompts_train))
  print('Cleaning Prompts... Storing Objects per prompt...')

  # Processing each prompt and updating annotation file for train set
  for idx, prompt in tqdm(enumerate(prompts_train)):
    cleaned, objects = clean_prompt(prompt) # Process prompt
    # update files and object list
    caption_data_train_file['annotations'][idx]['cleaned'] = cleaned
    caption_data_train_file['annotations'][idx]['objects'] = objects
    total_objects.update(set(objects))

    if (idx+1) % NUM_PROMPTS_INFO_DISPLAY == 0: # Display Info
      print()
      print()
      print('***INFO***')
      print('Number of prompts processed:', (idx+1))
      print('Number of Unique objects detected so far:', len(total_objects))
      num_objects_detected.append(len(total_objects))

    if (idx+1) % SAVE_AFTER == 0: # Save Results in json
      with open(os.path.join(PATH, 'Caption-Processing/train-captions-processed.json'), 'w') as outfile:
        outfile.write(json.dumps({'captions': caption_data_train_file['annotations'], 'num_objects': num_objects_detected}, indent=4))
        print('Saved.. Intermediate Results in train-captions-processed.json')

  if (idx-1) % NUM_PROMPTS_INFO_DISPLAY != 0: 
    print()
    print('***INFO***')
    print('Number of prompts processed:', (idx+1))
    print('Number of Unique objects detected so far:', len(total_objects))
    num_objects_detected.append(len(total_objects))

  with open(os.path.join(PATH, 'Caption-Processing/train-captions-processed.json'), 'w') as outfile: # Save Results in json
    outfile.write(json.dumps({'captions': caption_data_train_file['annotations'], 'num_objects': num_objects_detected}, indent=4))

  with open(os.path.join(PATH, 'Caption-Processing/train-objects.json'), 'w') as outfile: # Saving Total objects in json
    outfile.write(json.dumps({'objects': list(total_objects), 'num_objects': num_objects_detected}, indent=4))

  print('Saved and Finished Processing...')

except KeyboardInterrupt:
  print('Stopped Abruptly.. Saving.. Saved..')
  with open(os.path.join(PATH, 'Caption-Processing/train-captions-processed.json'), 'w') as outfile: # Save Results in json
    outfile.write(json.dumps({'captions': caption_data_train_file['annotations'], 'num_objects': num_objects_detected}, indent=4))

  with open(os.path.join(PATH, 'Caption-Processing/train-objects.json'), 'w') as outfile: # Saving Total objects in json
    outfile.write(json.dumps({'objects': list(total_objects), 'num_objects': num_objects_detected}, indent=4))

Starting...
Captions to be processed: 591753
Cleaning Prompts... Storing Objects per prompt...


503it [00:19, 24.31it/s]



***INFO***
Number of prompts processed: 500
Number of Unique objects detected so far: 507


1005it [00:39, 26.66it/s]



***INFO***
Number of prompts processed: 1000
Number of Unique objects detected so far: 767


...

265504it [2:48:57, 28.40it/s]



***INFO***
Number of prompts processed: 265500
Number of Unique objects detected so far: 10533


266005it [2:49:16, 28.22it/s]



***INFO***
Number of prompts processed: 266000
Number of Unique objects detected so far: 10538


266197it [2:49:23, 26.38it/s]

In [ ]:
print(total_objects)

In [ ]:
try:

  total_objects = set() # Stores the total number of distinct objects detected
  num_objects_detected = list() # Stores number of objects detected after processing some number of prompts iteratively

  print('Starting...')
  print('Captions to be processed:', len(prompts_val))
  print('Cleaning Prompts... Storing Objects per prompt...')

  # Processing each prompt and updating annotation file for val set
  for idx, prompt in tqdm(enumerate(prompts_val)):
    cleaned, objects = clean_prompt(prompt) # Process prompt
    # update files and object list
    caption_data_val_file['annotations'][idx]['cleaned'] = cleaned
    caption_data_val_file['annotations'][idx]['objects'] = objects
    total_objects.update(set(objects))

    if (idx+1) % NUM_PROMPTS_INFO_DISPLAY == 0: # Display Info
      print()
      print()
      print('***INFO***')
      print('Number of prompts processed:', (idx+1))
      print('Number of Unique objects detected so far:', len(total_objects))
      num_objects_detected.append(len(total_objects))

    if (idx+1) % SAVE_AFTER == 0: # Save Results in json
      with open(os.path.join(PATH, 'Caption-Processing/val-captions-processed.json'), 'w') as outfile:
        outfile.write(json.dumps({'captions': caption_data_val_file['annotations'], 'num_objects': num_objects_detected}, indent=4))
        print('Saved.. Intermediate Results in val-captions-processed.json')

  if (idx-1) % NUM_PROMPTS_INFO_DISPLAY != 0: 
    print()
    print('***INFO***')
    print('Number of prompts processed:', (idx+1))
    print('Number of Unique objects detected so far:', len(total_objects))

  with open(os.path.join(PATH, 'Caption-Processing/val-captions-processed.json'), 'w') as outfile: # Save Results in json
    outfile.write(json.dumps({'captions': caption_data_val_file['annotations'], 'num_objects': num_objects_detected}, indent=4))

  with open(os.path.join(PATH, 'Caption-Processing/val-objects.json'), 'w') as outfile: # Saving Total objects in json
    outfile.write(json.dumps({'objects': list(total_objects), 'num_objects': num_objects_detected}, indent=4))

  print('Saved and Finished Processing...')

except KeyboardInterrupt:
  print('Stopped Abruptly.. Saving.. Saved..')
  with open(os.path.join(PATH, 'Caption-Processing/val-captions-processed.json'), 'w') as outfile: # Save Results in json
    outfile.write(json.dumps({'captions': caption_data_val_file['annotations'], 'num_objects': num_objects_detected}, indent=4))

  with open(os.path.join(PATH, 'Caption-Processing/val-objects.json'), 'w') as outfile: # Saving Total objects in json
    outfile.write(json.dumps({'objects': list(total_objects), 'num_objects': num_objects_detected}, indent=4))

In [ ]:
print(total_objects)